# Programming Homework Week 6 (100 pts)

In this homework you will use a short corpus of data to build vectors for words and combinations of words, and ultimately build matrices for adjectives.

In [1]:
import numpy as np
import string
import matplotlib.pyplot as plt
import re

## Variables (3 pts)
You will need: 
1. A dictionary in which to store context words (as keys) and their counts (as values)
2. A dictionary of dictionaries to store adjectives (as keys), their arguments (as keys to the sub-dictionaries), and counts of adjective-argument occurrences (as values of the sub dictionary). Example: `adjective_counts = {'little':{'mouse':12, 'car':5}, 'big':{'elephant':6, 'car':7}}`. Choose two or more adjectives to learn (`little` and `big` are fine).
3. A list to store stopwords

In [2]:
# initialize empty dicts and lists
ctxtwords = dict()
adj_count = dict()
adj_count['little'] = {}
adj_count['big'] = {}
stopwords = []

## Stopwords (5 pts)
Open the file `stopwords.txt` and read in the stopwords, storing them in the list you created.

In [3]:
# open stopwords and store in a list
file = open('stopwords.txt','r')
for word in file:
    stopwords.append(word.split()[0])

FileNotFoundError: [Errno 2] No such file or directory: 'stopwords.txt'

### Create a translation to remove punctuation (2 pts)
See this week's practice exercises for details

In [ ]:
# translation table
table = str.maketrans(dict.fromkeys(string.punctuation,None))

## Collecting vocabulary counts (15 points)

1. Open the file `story.txt` and read it in one line at a time. (1 pt)
2. For each line, use the function `re.split()` to split the line on non-alphanumeric characters. (2 pts)
3. For each word in the list of strings, process the word to remove punctuation and put it all in lowercase. Check whether it is already in your dictionary of context words, and if so, increment the count. If not, add it to your dictionary of context words with count 1 (4 pts)
4. Also, check whether the word is one of the adjectives you will be learning. If it is, then look at the next word in the line and either increment the count of that word in the dictionary of that adjective, or add the word to the dictionary for that adjective with count 1. (6 pts) 
5. Note that the adjective may be the last word in the line. In that case, you will have to temporarily store it until the next line is read in. (2 pts)

In [ ]:
# open file and initialize list and 
story = open('story.txt','r')
temp_word = (0,"")
corpus = []

# for every word in the file strip it and clean it
for line in story:
    split_line = re.split('\W+',line)
    for word in split_line:
        clean_word = word.translate(table).lower()
        if clean_word != "" and clean_word not in stopwords:
            corpus.append(clean_word)
            
# this will be used if the adjective is at the end of the line           
for i in range(len(corpus)):
    if temp_word[0] == 1 and clean_word != "":
        if clean_word in adj_count[temp_word[1]]:
            adj_count[temp_word[1]][clean_word] += 1
        else: 
            adj_count[temp_word[1]][clean_word] = 1
        temp_word = (0,"")
    # filter out empty strings
    if corpus[i] != "":
        #increment the context words
        if corpus[i] in ctxtwords:
            ctxtwords[corpus[i]] += 1
        else:
            ctxtwords[corpus[i]] = 1
    
    # check if the word is big or little
    if corpus[i] in adj_count:
        next_clean_word = corpus[i+1]
        # check if the next word is not empty
        if next_clean_word != "":
            # add the next word to the adjective count
            if next_clean_word in adj_count[corpus[i]]:
                adj_count[corpus[i]][next_clean_word] += 1
            else:
                adj_count[corpus[i]][next_clean_word] = 1
        else:
            temp_word = (1,clean_word) 
            

## Collecting a list of target words (5 pts)
Create a list of context words from the dictionary you built from the corpus. (1 pt)

Create a list of target words, encoded as strings, that consists of all the context words and all the adjective-noun combinations you found. (4 pts)

Example:

`target_word_list = ['car', 'elephant', 'mouse', 'little', 'big', 'little car', 'little mouse', 'big car', 'big mouse']`

In [ ]:
# initialize empty lists
ctxtwordslist = []
target_word_list = []

# add all context words to the lists
for key in ctxtwords:
    ctxtwordslist.append(key)
    target_word_list.append(key)

# add the adjective-noun combinations to the target word list
for word in ctxtwordslist:
    if word in adj_count['little']:
        target_word_list.append('little '+ word)
        target_word_list.append(word)
    if word in adj_count['big']:
        target_word_list.append('big ' + word)
        target_word_list.append(word)
        
target_word_list = list(set(target_word_list))

## Collecting cooccurrence counts (25 points)
We cycle through the corpus again and collect cooccurrence counts for each target word (or phrase).

1. Set a variable for the width of the window and initialise a list of empty strings of twice the length of the window (1 pt)
2. Initialise a matrix of zeros with number of rows equal to the number of target words and phrases, and number of columns equal to the number of context words (1 pt)
3. Open the file `story.txt` and read it in one line at a time. (1 pt)
4. For each line, split it into a list of strings by breaking on non-alphanumeric characters. (2 pts)
5. Process each word in the list to ensure that it is all in lowercase, contains no punctuation, is not a stopword, and is not empty. (2 pts)
6. For each word in the list:
    - append it to the current window
    - check whether the central word in the window is in the set of target words. 
    - If it is, then iterate over the context words in the current window and increment the entry in the cooccurrence matrix corresponding to (target word, context word) by 1, leaving out the target word itself. (8 pts)
    - Check whether the word in front of the target word is one of your adjectives (i.e., the target word is the word in the centre of the context window, and you are checking whether the word in front of it is an adjective)
    - If it is, then increment the entry in the coocurrence matrix accordingly (8 pts)
7. Discard the first element of the current window (2 pts)


In [ ]:
# set parameters and make the matrix M
window_width = 2
window = [''] * (window_width*2+1)
M = np.zeros((len(target_word_list),len(ctxtwordslist)))

# since our corpus is one list, we have to use the if statements because of the end of the list
for i in range(len(corpus)-1):
    if i == 0:
        window[2] = corpus[i]
        window[3] = corpus[i+1]
        window[4] = corpus[i+2]
    elif i == 1:
        window[1] = corpus[i-1]
        window[2] = corpus[i]
        window[3] = corpus[i+1]
        window[4] = corpus[i+2]
    elif i == len(corpus)-1:
        window[0] = corpus[i-2]
        window[1] = corpus[i-1]
        window[2] = corpus[i]
        window[3] = corpus[i+1]
        window[4] = ""
    elif i == len(corpus)-2:
        window[0] = corpus[i-2]
        window[1] = corpus[i-1]
        window[2] = corpus[i]
        window[3] = ""
        window[4] = ""
    else:
        window[0] = corpus[i-2]
        window[1] = corpus[i-1]
        window[2] = corpus[i]
        window[3] = corpus[i+1]
        window[4] = corpus[i+2]
    # check if the word in the middle of the window is in the target word list
    if window[2] in target_word_list: 
        # get the word its position
        pos1 = target_word_list.index(window[2])
        # increment the context words from the window
        for word in window:
            if word != "":
                pos2 = ctxtwordslist.index(word)
                M[pos1][pos2]+=1
        # increment the adj_noun combinations as well
        if window[1] in adj_count:
            pos1 = target_word_list.index(window[1]+' '+window[2])
            M[pos1][ctxtwordslist.index(window[1])] += 1
            M[pos1][ctxtwordslist.index(window[2])] += 1

## Calculating PMI (15 pts)
Calculate a version of the PMI that is based only on counts (no logarithm)

Recall that the equation is:

$$ PMI(t, c) = \frac{count(t, c)\times total}{count(t)count(c)}$$

where $count(t,c)$ is the number of times that word $t$ cooccurs with word $c$, $count(t)$ is the total number of times that word $t$ occurs in the text, $count(c)$ is the total number of times that word $t$ occurs in the text, and $total$ is the total number of words in the text.

In [ ]:
# initialize empty matrix
PMI = np.zeros((len(target_word_list),len(ctxtwordslist)))

# loop through all rows and column
for i in range(len(M)):
    for j in range(len(M[i])):
        # if there is a " " in the word then it is a adj-noun combination
        if " " in target_word_list[i]:
            adj = target_word_list[i].split(" ")[0]
            nn = target_word_list[i].split(" ")[1]
            countw1 = adj_count[adj][nn]
        # otherwise just get the count for a single word
        else:
            countw1 = ctxtwords[target_word_list[i]]
        countw2 = ctxtwords[ctxtwordslist[j]]
        # calculate PMI
        PMI[i][j] = (M[i][j]*len(corpus))/(countw1*countw2)

## Dimensionality reduction (10 pts)
To reduce the number of dimensions we will use SVD. Recall that the principle behind SVD is that any $m \times n$ matrix $M$ can be written as 

$$ M = U \Sigma V^* $$

where $U$ is a $m \times m$ matrix of orthonormal eigenvectors  for  $MM^*$, $V$ is a $n \times n$ matrix of orthonormal eigenvectors  for  $M^*M$, and $\Sigma$ is a diagonal matrix of singular values.

In `numpy.linalg.svd`, the singular values are returned in decreasing order. This means that we can select the first $k$ singular values in order to reduce the dimensionality to $k$

1. Use the function `numpy.linalg.svd` on the matrix of PMI values to obtain the matrix $U$, the list of singular values $s$ and the matrix $V^*$. (2 pts)
2. Plot the singular values and use the shape of the plot to decide to many dimensions `k` to reduce to. State how many dimensions you are reducing to and why. (3pts)
3. Shorten the list of singular values to your desired length `k`. Embed the list of singular values in a diagonal matrix $\Sigma_k$ of shape `(k, k)` (2 pts)
4. Now modify $U$ to $U_k$ and $V^*$ to $V^*_k$ so that when you multiply together $U_k \Sigma_k V^*_k$ you result in a matrix $PMI_k$ of shape $(m, k)$, where $m$ is the number of target words and phrases. I.e., you now have a list of vectors of length $k$, one for each target word or phrase. (3 pts)

In [ ]:
# compute SVD
U, S, V = np.linalg.svd(PMI)

# show the singular values
plt.plot(S)
plt.show()

# reduce dimensionality
S = S[:450]

# create the diagonal matrix
D = np.diag(S)

# adjust U
UR = U[:len(target_word_list),:450]

MR = UR.dot(D)

## Building your data (10 pts)

For *each* of the adjectives you have chosen, build *two* matrices. One should consist of the vectors for the arguments of the adjective, and the other should consist of the vectors for the adjective-noun combinations.

Example: 
- For `little`, you will have one matrix with the vectors for `mouse` and `car`, and one matrix with the vectors for `little mouse` and `little car`.
- For `big` you will have one matrix with the vectors for `elephant` and `car` and one matrix with the vectors for `big elephant` and `big car`


In [ ]:
# initialize empty matrices
l_vec1 = np.zeros((450,))
l_vec2 = np.zeros((450,))

# create the matrices 
for key in adj_count['little']:
    l_vec1 = np.vstack([l_vec1, MR[target_word_list.index(key)]])
    l_vec2 = np.vstack([l_vec2, MR[target_word_list.index('little '+key)]])

# remove the first empty row
l_vec1 = l_vec1[1:47]    
l_vec2 = l_vec2[1:47]

# initialize empty matrices
b_vec1 = np.zeros((450,))
b_vec2 = np.zeros((450,))

# create the matrices 
for key in adj_count['big']:
    b_vec1 = np.vstack([b_vec1, MR[target_word_list.index(key)]])
    b_vec2 = np.vstack([b_vec2, MR[target_word_list.index('big '+key)]])

# remove the first empty row
b_vec1 = b_vec1[1:47]    
b_vec2 = b_vec2[1:47]

## Learning adjective matrices (5 pts)
For each of the adjectives you have chosen, infer an adjective matrix from the nouns vectors and the adjective-noun combinations using linear regression (you can use the function `np.linalg.pinv()` to implement the linear regression) 

In [ ]:
# the adjective matrix can be derived by multiplying the pseudo-inverse of the noun-matrix with the noun-adjective matrix n
adj1 = np.linalg.pinv(l_vec1).dot(l_vec2)
adj2 = np.linalg.pinv(b_vec1).dot(b_vec2)